# Export LDA Embeddings

    Creation date : 2020-04-06 (Monday)
    Creator       : Stanislav Schmidt <stanislav.schmidt@epfl.ch>

In [12]:
from pathlib import Path
import joblib

import pandas as pd
import en_core_sci_md

In [50]:
metadata_path = Path("/raid/covid19_kaggle-data/preproccessed_data")
model_path = Path("/raid/covid19_kaggle-data/stan/1_topic_modeling_lda")
export_path = Path("/raid/covid19_kaggle-data/exports")

In [1]:
nlp = en_core_sci_md.load(disable=["tagger", "parser", "ner"])
nlp.max_length = 2_000_000  # maximal number of characters in text

In [4]:
def spacy_tokenizer(sentence):
    return [token.lemma_ for token in nlp(sentence) if not (
        token.like_url or
        token.like_email or
        token.like_num or
        token.is_stop or
        token.is_punct or
        token.is_space or
        token.is_bracket or
        token.is_punct or
        len(token)==1)]

In [13]:
df_docs = pd.read_pickle(metadata_path / "v4_json_metadata_v2.pkl")
df_docs = df_docs
df_docs.shape

(29135, 18)

In [14]:
df_docs.head()

,title,abstract,text,sections,ref_captions,all_text,source_x,doi,pmcid,pubmed_id,license,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
sha,,,,,,,,,,,,,,,,,,
0001418189999fea7f7cbe3e82703d71c85a6fe5,Absence of surface expression of feline infect...,Feline infectious peritonitis virus (FIPV) pos...,Feline infectious peritonitis (FIP) is a fatal...,Results\nIsolation of FIPV positive cells\nCat...,Surface expression of the monocyte marker (A);...,Feline infectious peritonitis virus (FIPV) pos...,Elsevier,10.1016/j.vetmic.2006.11.026,NaN,17188823.0,els-covid,2007-03-31,"Cornelissen, E.; Dewerchin, H.L.; Van Hamme, E...",Veterinary Microbiology,NaN,NaN,True,custom_license
00016663c74157a66b4d509d5c4edffd5391bbe0,,,Viruses are increasingly recognised as pathoge...,Burden of disease studies\nAn integrated molec...,Scheme for foodborne virus tracking. Virus det...,\nViruses are increasingly recognised as patho...,Elsevier,10.1533/9781845691394.1.77,NaN,NaN,els-covid,2006-12-31,"Duizer, E.; Koopmans, M.",Emerging Foodborne Pathogens,NaN,NaN,True,custom_license
0005d253951fedc237715a37db147032eea28912,,,Synthesized by William Prusoff in the late 195...,\nLuis Mené ndez-Arias,//dx.doi.org/10.1016/j.coviro.2014.08.002 1879...,\nSynthesized by William Prusoff in the late 1...,Elsevier,10.1016/j.coviro.2014.08.002,NaN,25155454.0,els-covid,2014-10-31,"Menéndez-Arias, Luis; Richman, Douglas D",Current Opinion in Virology,NaN,NaN,True,custom_license
000affa746a03f1fe4e3b3ef1a62fdfa9b9ac52a,Correlation between antimicrobial consumption ...,Objectives: This study was conducted to invest...,The incidence of health-care-associated infect...,Antimicrobial agents and consumption\nCorrelat...,(A) Rates of methicillin-resistant Staphylococ...,Objectives: This study was conducted to invest...,Elsevier,10.1016/j.jmii.2013.10.008,NaN,24388582.0,els-covid,2015-08-31,"Lai, Chih-Cheng; Chu, Chen-Chen; Cheng, Aristi...","Journal of Microbiology, Immunology and Infection",NaN,NaN,True,custom_license
000b7d1517ceebb34e1e3e817695b6de03e2fa78,Supplementary Information An eco-epidemiologic...,,- Figure S1 : Phylogeny of all sequences belon...,,Table S4. Bats cytochrome b sequences data set...,\n- Figure S1 : Phylogeny of all sequences bel...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
vectorizer = joblib.load(model_path / "tf_vectorizer.csv")
lda = joblib.load(model_path / "lda.csv")

In [36]:
def embed(text):
    if isinstance(text, str):
        text = [text]
    tf = vectorizer.transform(text)
    embedding = lda.transform(tf)
    
    return embedding

In [56]:
embeddings = embed(df_docs['text'])

In [57]:
df_embeddings = pd.DataFrame(embeddings, index=df_docs.index)

In [58]:
df_embeddings.to_csv(export_path / "lda_embeddings.csv")